In [13]:
# --- BAGIAN 1: Load & Explorasi Data ---
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target
nama_fitur = iris.feature_names
nama_target = iris.target_names

# Membuat DataFrame utama
df = pd.DataFrame(X, columns=nama_fitur)
df['target'] = y

# 1. Menampilkan Jumlah Data & Fitur
print("=== 1. INFO DATASET ===")
print(f"Jumlah Data (Baris) : {df.shape[0]}")
print(f"Jumlah Fitur (Kolom): {df.shape[1] - 1}")

# 2. Menampilkan Nama Fitur
print(f"\n=== 2. NAMA FITUR ===")
# Menampilkan sebagai list bullet agar rapi
for fitur in nama_fitur:
    print(f"- {fitur}")

# 3. Distribusi Kelas Target (MODIFIKASI MENJADI TABEL)
print("\n=== 3. DISTRIBUSI KELAS TARGET ===")

# Menghitung jumlah
counts = df['target'].value_counts().sort_index()

# Membuat DataFrame Ringkasan (Tabel)
tabel_distribusi = pd.DataFrame({
    'Kode Kelas': counts.index,
    'Nama Spesies': [nama_target[i] for i in counts.index], # Ambil nama berdasarkan index
    'Jumlah Data': counts.values
})

# Print tabel tanpa index bawaan pandas agar lebih bersih
print(tabel_distribusi.to_string(index=False))

=== 1. INFO DATASET ===
Jumlah Data (Baris) : 150
Jumlah Fitur (Kolom): 4

=== 2. NAMA FITUR ===
- sepal length (cm)
- sepal width (cm)
- petal length (cm)
- petal width (cm)

=== 3. DISTRIBUSI KELAS TARGET ===
 Kode Kelas Nama Spesies  Jumlah Data
          0       setosa           50
          1   versicolor           50
          2    virginica           50


In [14]:
# --- BAGIAN 2: Split & Scaling ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Split Data (70% Training, 30% Testing)
# random_state=42 digunakan agar hasil acakan tetap sama setiap kali dijalankan
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Feature Scaling (StandardScaler)
scaler = StandardScaler()

# PENTING:
# .fit_transform() hanya pada TRAINING set (agar model belajar statistik dari data latih saja)
# .transform() pada TESTING set (menggunakan statistik dari data latih)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n=== 2. PREPROCESSING ===")
print(f"Data Training : {X_train.shape[0]} baris")
print(f"Data Testing  : {X_test.shape[0]} baris")
print("Status: Data berhasil di-scale dengan StandardScaler.")


=== 2. PREPROCESSING ===
Data Training : 105 baris
Data Testing  : 45 baris
Status: Data berhasil di-scale dengan StandardScaler.


In [15]:
# --- BAGIAN 3: Modeling & Evaluasi ---
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score

# 1. Implementasi KNN (k=5)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# 2. Prediksi Data Test & Hitung Metrik
y_pred = knn.predict(X_test_scaled)

# Menggunakan 'macro' karena ini klasifikasi multi-kelas (3 kelas)
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("\n=== 3. EVALUASI MODEL ===")
print(f"Accuracy : {acc:.2f}")
print(f"Recall   : {rec:.2f}")
print(f"F1-Score : {f1:.2f}")

# 3. Input Manual (Interaktif)
print("\n--- TEST INPUT MANUAL ---")
print("Masukkan nilai fitur untuk prediksi spesies bunga.")

try:
    input_user = []
    # Loop meminta input user sesuai nama fitur
    for fitur in nama_fitur:
        nilai = float(input(f"Masukkan {fitur}: "))
        input_user.append(nilai)

    # Ubah ke array 2D karena scikit-learn butuh format matriks
    data_baru = np.array([input_user])

    # PENTING: Lakukan scaling pada input manual menggunakan scaler yang sudah dilatih
    data_baru_scaled = scaler.transform(data_baru)

    # Prediksi
    hasil_idx = knn.predict(data_baru_scaled)[0]
    hasil_label = nama_target[hasil_idx]

    print(f"\n[HASIL] Data {input_user} diprediksi sebagai: {hasil_label.upper()}")

except ValueError:
    print("\n[ERROR] Masukkan hanya angka!")


=== 3. EVALUASI MODEL ===
Accuracy : 1.00
Recall   : 1.00
F1-Score : 1.00

--- TEST INPUT MANUAL ---
Masukkan nilai fitur untuk prediksi spesies bunga.
Masukkan sepal length (cm): 2
Masukkan sepal width (cm): 3
Masukkan petal length (cm): 2
Masukkan petal width (cm): 1

[HASIL] Data [2.0, 3.0, 2.0, 1.0] diprediksi sebagai: SETOSA
